**<font color = black size=6>实验五:决策树</font>**

**<font color = blue size=4>第一部分:函数介绍</font>**

In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

1)Counter类的使用

In [2]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
#使用Counter类对数组第2列进行遍历
counter = Counter(x[:,1])
#第2列中有1个132和2个133，输出该counter对象可以统计这列的数值情况，便于之后的统计
print(counter)
#因为第2列中没有为0的值，所以返回0
print(counter[0])
#因为第2列中有2个133，所以返回2
print(counter[133])
#一般的字典操作方法都能在该类中使用，例如可以通过values函数返回该列的非重复值的个数，方便对某列的非重复值的个数进行查看
print(counter.values())
#可以输出所有非重复值
print(list(counter))

Counter({133: 2, 132: 1})
0
2
dict_values([2, 1])
[133, 132]


2)使用numpy中的unique实现非重复值的提取

In [3]:
x=np.array([[0,133,1],[0,132,0],[0,133,0]])
a=np.unique(x[:])
a1=np.unique(x[:,1])
print(a,a1)

[  0   1 132 133] [132 133]


**<font color = blue size=4>第二部分:实验任务</font>**

**<font color = blue size=3>1) 任务一</font>**

任务一要求完成不同量化标准(信息增益, 信息增益率, 基尼系数)下的结点划分函数

<span style="color:purple">该数据集(train_titanic.csv)为分类数据集，为泰坦尼克号的部分乘客信息以及最后是否生还。包括了四个属性特征以及一个标签(即为Survived,代表是否生还),属性特征分别为Sex(性别)，sibsp(堂兄弟妹个数)，Parch(父母与小孩的个数)，Pclass(乘客等级)  
其中该数据集无缺失值和异常值，且所有连续变量已自动转换为离散变量,标签(Survived)也自动转变为离散变量0和1，所以你无需进行数据预处理，可以直接使用该数据集。</span>

In [4]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

<span style="color:purple">1) 使用pandas库将训练数据集'train_titanic.csv'载入到Dataframe对象中</span>

In [5]:
#----your code here

train_titanic = pd.read_csv("train_titanic.csv")
print(train_titanic)

test_titanic = pd.read_csv("test_titanic.csv")
print(test_titanic)


      Sex  sibsp  Parch  Pclass  Survived
0       0      1      0       3         0
1       1      1      0       1         1
2       1      0      0       3         1
3       1      1      0       1         1
4       0      0      0       3         0
...   ...    ...    ...     ...       ...
1004    0      0      0       3         0
1005    1      0      0       1         0
1006    0      0      0       3         0
1007    0      0      0       3         0
1008    0      1      1       3         0

[1009 rows x 5 columns]
     Sex  sibsp  Parch  Pclass  Survived
0      0      0      0       1         0
1      1      1      0       1         1
2      1      0      0       2         0
3      1      1      0       1         1
4      1      1      1       2         0
..   ...    ...    ...     ...       ...
197    1      0      0       3         1
198    0      1      0       3         0
199    0      0      0       1         0
200    1      0      2       2         1
201    0      0     

<span style="color:purple">2) 编写函数，给定任何标签数组计算其信息熵  
    输入：标签数组  
    输出：该数组对应的信息熵  
    计算信息熵公式:
某数组包含K个不同的取值，样本为第k(k=1,2,...,K)个值的数量所占比例为p_k,则其信息熵为$$Ent=-\sum_{k=1}^K p_k log_2 p_k$$</span>
    
    
<span style="color:purple">例:  
    输入:[[0],[1]]   
    输出:(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)+(-$\frac{1}{2}$ $log_2$ $\frac{1}{2}$)=1</span>

In [6]:
# 计算标签数组的信息熵
def entropy(label):
    unique_classes, counts = np.unique(label, return_counts=True)
    total_samples = len(label)
    entropy_value = 0
    
    for count in counts:
        p_k = count / total_samples
        entropy_value -= p_k * np.log2(p_k)

    return entropy_value

<span style="color:purple">3) 编写函数，函数功能为将所给的数据集按照指定维度的特征进行划分为若干个不同的数据集  
    【输入】：特征集合，标签集合，指定维度  
    【输出】：划分后所得到的子树属性集合，子树标记集合</span>

<span style="color:purple">例子:  
    【输入】:特征集合:[[0,0,0],[0,0,1],[1,0,2]]  
    标签集合:[[0],[1],[2]]  
    指定维度:0</span>  
    
<span style="color:purple">【输出】:[[0,0,0],[0,0,1]]和[[1,0,2]]  
    [[0],[1]]和[[2]]  
    tips:即将特征按其第0维度进行划分，由于第0维特征有0和1两个不同的数值，所以特征集合划分为[[0,0,0],[0,0,1]]和[[1,0,2]]，标签集合划分为[[0],[1]]和[[2]]</span>

In [7]:

def split(features, labels, d):
    split_feature = {}
    split_label = {}

    features = np.array(features)
    labels = np.array(labels)

    for i in range(len(features)):
        # 获取指定维度上的特征值
        feature = features[i, d]
        label = labels[i, 0]

        # 如果特征值已在split_feature中,则添加到对应子集中,否则创建一个新子集
        if feature in split_feature:
            split_feature[feature].append(features[i])
            split_label[feature].append(label)
        else:
            split_feature[feature] = [features[i]]
            split_label[feature] = [label]

    # 结果转换为numpy
    split_feature = list(split_feature.values())
    split_label = list(split_label.values())

    return split_feature, split_label


<span style="color:purple">4) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该特征集合中信息增益(使用ID3算法中的公式计算)【最大】的特征  
    输入：特征集合，标签集合  
    输出：该次划分的最佳信息增益值，最佳划分维度  
    计算信息增益公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，其中值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征值对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$
该函数中你需要计算出每个特征的信息增益并输出，然后返回最佳的信息增益值和对应的特征的维数</span>

In [8]:
# 计算考虑属性信息后的条件熵、给定维度的条件下的信息熵
def conditional_entropy(x_data, y_label, dimension):
    
    # 获取某个特征的所有可能取值
    unique_values = np.unique(x_data[:, dimension])
    total_samples = len(y_label)
    cond_entropy_value = 0

    for value in unique_values:
        # 获取相应特征的标签值
        sub_indices = np.where(x_data[:, dimension] == value)[0]
        sub_labels = y_label[sub_indices]
        
        # 计算占比
        sub_samples = len(sub_labels)
        p = sub_samples / total_samples
        
        # 计算条件熵
        sub_entropy = entropy(sub_labels)
        cond_entropy_value += p * sub_entropy

    return cond_entropy_value

# 选择信息增益最大的特征输出
def one_split_ID3(x_data, y_label):
    num_features = x_data.shape[1]
    total_entropy = entropy(y_label)
    best_info_gain = 0
    best_dimension = None

    for dimension in range(num_features):
        cond_entropy = conditional_entropy(x_data, y_label, dimension)
        info_gain = total_entropy - cond_entropy

        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_dimension = dimension
        
    return best_info_gain, best_dimension

<span style="color:purple">5) 编写函数，函数功能为进行【一次】决策树的结点划分，遍历找出该特征集合中信息增益率(使用C4.5算法中的公式计算)【最大】的特征  
    输入：特征集合，标签集合  
    输出：最佳划分的信息增益率值，对应的划分维度  
    计算信息增益率公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征类型feature,该特征包含V个不同的取值，值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^V|D^v|=|D|)$,该特征本身的信息熵为Ent(feature),则该特征值对应的信息增益率为$$Gain\_ratio(D,feature)=\frac{Gain(D,feature)}{Ent(feature)}$$
该函数中你需要输出每个特征的信息增益率，之后返回最佳的信息增益率和对应的特征维数</span>

In [9]:
def IV(x_data, y_label, dimension):
    # 获取某个特征的所有可能取值
    unique_values = np.unique(x_data[:, dimension])
    total_samples = len(y_label)
    iv = 0.0

    for value in unique_values:
        # 获取相应特征的标签值
        sub_indices = np.where(x_data[:, dimension] == value)[0]
        sub_labels = y_label[sub_indices]
        
        # 计算占比
        sub_samples = len(sub_labels)
        p = sub_samples / total_samples
        
        # 计算条件熵
        iv -= p * math.log2(p)
        
    return iv

def one_split_C4_5(x_data, y_label):
    num_features = x_data.shape[1]
    total_entropy = entropy(y_label)
    best_gain_rate = 0.0
    best_dimension = None
    
    for dimension in range(num_features):
        cond_entropy = conditional_entropy(x_data, y_label, dimension)
        info_gain = total_entropy - cond_entropy
        iv = IV(x_data, y_label,dimension)
        gain_rate = info_gain / iv

        if gain_rate > best_gain_rate:
            best_gain_rate = gain_rate
            best_dimension = dimension

    return best_gain_rate, best_dimension

<span style="color:purple">6) 编写函数，进行【一次】决策树的结点划分，遍历找出该特征集合中基尼系数(使用CART算法中的公式计算)最小的特征以及最佳的划分值  
    输入：特征集合，标签集合  
    输出：最佳的基尼系数，对应的划分维度，最佳划分值  
    计算基尼系数公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,该集合中标签类别总共有K类，第k类样本所占比例为$p_k$(k=1,2,..,K),则该数据集对应的基尼系数为
    $$Gini(D)=1-\sum_{k=1}^K{p_k}^2$$  
    而取其中一个特征feature，选定该特征的一个值value，根据该特征的值是否为value将数据集分为两个子集$D_1$和$D_2$,则该特征对应的基尼系数为$$Gini\_index(D,feature)=\frac{|D_1|}{|D|}Gini(D_1)+\frac{|D_2|}{|D|}Gini(D_2)$$
该函数中你需要遍历每一列特征，找出每列中的非重复值，计算出每个非重复值的基尼系数，返回最小的基尼系数、对应的特征维数和非重复值（分类值）。</span>

In [10]:
def Gini(y):
    # 计算基尼系数
    unique_classes, counts =np.unique(y, return_counts=True)
    total_samples = len(y)
    gini = 1.0
    for count in counts:
        p = count / total_samples
        gini -= p**2
    return gini

def one_split_CART(x_data, y_label):
    best_gini = 1.0
    best_dimension = None
    best_value = None

    for dimension in range(x_data.shape[1]):
        unique_values = np.unique(x_data[:, dimension])
        for value in unique_values:
            # 根据特征值划分数据集
            left = x_data[:, dimension] <= value
            right = x_data[:, dimension] > value

            # 计算各部分的基尼系数
            left_gini = Gini(y_label[left])
            right_gini = Gini(y_label[right])

            # 计算加权基尼系数
            gini = (left_gini * np.sum(left) + right_gini * np.sum(right)) / len(y_label)

            # 判断是否为最佳
            if gini < best_gini:
                best_gini = gini
                best_dimension = dimension
                best_value = value
    
    return best_gini, best_dimension, best_value

<span style="color:purple">7) 应用之前你在第4、5、6个部分编写的三个函数，在训练数据集'train_titanic.csv'上依次使用这些函数进行【一次】结点划分，并输出对应的最佳特征维数以及相应的信息增益值/信息增益率/(基尼系数和分类值)</span>

In [11]:
#----your code here
x_data = train_titanic.drop(columns=['Survived']).values
y_label = train_titanic['Survived'].values

best_info_gain, best_dimension_ID3 = one_split_ID3(x_data, y_label)
best_gain_rate, best_dimension_C4_5 = one_split_C4_5(x_data, y_label)
best_gini, best_dimension_CART, best_value = one_split_CART(x_data, y_label)

print("信息增益的最佳特征维度为：", best_dimension_ID3)
print("信息增益率的最佳特征维度为：", best_dimension_C4_5)
print("基尼系数最佳特征维度为：", best_dimension_CART)

print("信息增益值", best_info_gain)
print("信息增益率", best_gain_rate)
print("基尼系数", best_gini)
print("分类值", best_value)


信息增益的最佳特征维度为： 0
信息增益率的最佳特征维度为： 0
基尼系数最佳特征维度为： 0
信息增益值 0.10750711887455178
信息增益率 0.11339165967945304
基尼系数 0.2964915724641573
分类值 0


**<font color = blue size=3>2) 任务二</font>**

任务二承接任务一，用【ID3】算法实现一棵完整的决策树。

<span style="color:purple">1) 整理任务一的代码，根据实验二要求做适当修改来编写函数，【从剩余的特征集A中】寻找使得信息增益最大的特征   
    输入：数据集D、剩余的特征集A    
    输出：最佳划分的特征维数    
    计算信息增益公式:  
    某数据集D有若干特征值以及对应的标签值，其总样本大小为|D|,这里取其中一个特征feature,该特征包含V个不同的取值，特征值为第v(v=1,2,...,V)个值的样本数量为|$D^v$|$(\sum_{v=1}^VD^v=|D|)$,则该特征对应的信息增益为$$Gain(D,feature)=Ent(D)-\sum_{v=1}^K \frac{|D^v|}{D} Ent(D^v)$$</span>

In [12]:
def best_split(D, A):
    best_dimension = None
    best_info_gain = 0
    
    for dimension in A:
        x_data = D[:, dimension].reshape(-1, 1)
        info_gain, _ = one_split_ID3(x_data, D[:, -1])
        
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_dimension = dimension

    return best_dimension


<span style="color:purple">2) 完成DTree类中的TreeGenerate、train函数以完成决策树的构建。并完成DTree类中的predict函数来用构建好的决策树来对测试数据集进行预测并输出预测准确率。</span>

In [13]:
# 树结点类
class Node:
    def __init__(self, isLeaf=True, label=-1, feature_index=-1):
        self.isLeaf = isLeaf                # isLeaf表示该结点是否是叶结点
        self.label = label                  # label表示该叶结点的label(当结点为叶结点时有用)
        self.feature_index = feature_index  # feature_index表示该分支结点的划分特征的序号(当结点为分支结点时有用)
        self.children = {}                  # children表示该结点的所有孩子结点，dict类型，方便进行决策树的搜索
        
    def addNode(self, val, node):
        self.children[val] = node           #为当前结点增加一个划分特征的值为val的孩子结点

In [14]:
# 决策树类
class DTree:
    def __init__(self):
        self.tree_root = None       # 决策树的根结点
        self.possible_value = {}    # 用于存储每个特征可能的取值
    
    '''
    TreeGenerate函数用于递归构建决策树,伪代码参照课件中的“Algorithm 1 决策树学习基本算法”
    
    '''    
    
    def TreeGenerate(self, D, A):
        node = Node(isLeaf=False)

        # if D中样本全属于同一类别C then
        if len(np.unique(D[:, -1])) == 1:
            node.isLeaf = True
            node.label = D[0, -1]  # 叶结点的类别为D中唯一的类别
            return node
        # end if

        # if A = Ø OR D中样本在A上取值相同 then
        if len(A) == 0 or len(np.unique(D[:, list(A)])) == 1:
            node.isLeaf = True
            node.label = np.argmax(np.bincount(D[:, -1]))  # 叶结点的类别为D中样本最多的类
            return node
        # end if

        # 从A中选择最优划分特征a_star
        a_star = best_split(D, A)
        node.feature_index = a_star  # 设置分支节点的划分特征

        # for a_star 的每一个值a_star_v do
        for a_star_v in self.possible_value[a_star]:
            D_v = D[D[:, a_star] == a_star_v]

            if len(D_v) == 0:
                child_node = Node(isLeaf=True, label=np.argmax(np.bincount(D[:, -1])))
            else:
                A_new = A - {a_star}
                child_node = self.TreeGenerate(D_v, A_new)

            node.addNode(a_star_v, child_node)
        # end for

        return node
      
 
    '''
    train函数可以做一些数据预处理(比如Dataframe到numpy矩阵的转换,提取属性集等),并调用TreeGenerate函数来递归地生成决策树
    
    '''
    
    def train(self, D):
        # 将Dataframe对象转换为numpy矩阵（也可以不转，自行决定做法）
        D = np.array(D) 
        
        # 特征集A
        A = set(range(D.shape[1] - 1))
        
        #记下每个特征可能的取值
        for every in A:
            self.possible_value[every] = np.unique(D[:, every])

        # 递归地生成决策树，并将决策树的根结点赋值给self.tree_root
        self.tree_root = self.TreeGenerate(D, A)
        
        return

    
    '''
    predict函数对测试集D进行预测,并输出预测准确率(预测正确的个数 / 总数据数量)
    
    '''

    def predict(self, D):
        D = np.array(D)
        correct_predictions = 0

        for i in range(len(D)):
            current_node = self.tree_root

            while not current_node.isLeaf:
                feature_index = current_node.feature_index
                feature_value = D[i, feature_index]

                if feature_value in current_node.children:
                    current_node = current_node.children[feature_value]
                else:
                    # 若测试数据包含了训练集中未出现的特征值，做一种简单的处理，将其分类到出现次数最多的类别
                    current_node = Node(isLeaf=True, label=np.argmax(np.bincount(D[:, -1])))

            predicted_label = current_node.label
            actual_label = D[i, -1]

            if predicted_label == actual_label:
                correct_predictions += 1

        accuracy = correct_predictions / len(D)
        return accuracy


In [15]:
#----your code here

# 构建决策树
decision_tree = DTree()

# 训练决策树
decision_tree.train(train_titanic)

# 利用构建好的决策树对测试数据集进行预测，输出预测准确率
accuracy = decision_tree.predict(test_titanic)

print(f"预测准确率: {accuracy:.3f}")


预测准确率: 0.837


In [16]:
#展示生成的决策树结构
def display_tree(node, indent=''):
    if node.isLeaf:
        print(indent + "Leaf Node: label =", node.label)
    else:
        print(indent + "Branch Node: feature_index =", node.feature_index)
        for value, child in node.children.items():
            print(indent + "|--> Child Value:", value)
            display_tree(child, indent + "   ")


display_tree(decision_tree.tree_root)



Branch Node: feature_index = 0
|--> Child Value: 0
   Branch Node: feature_index = 2
   |--> Child Value: 0
      Branch Node: feature_index = 3
      |--> Child Value: 1
         Branch Node: feature_index = 1
         |--> Child Value: 0
            Leaf Node: label = 0
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
         |--> Child Value: 4
            Leaf Node: label = 0
         |--> Child Value: 5
            Leaf Node: label = 0
         |--> Child Value: 8
            Leaf Node: label = 0
      |--> Child Value: 2
         Branch Node: feature_index = 1
         |--> Child Value: 0
            Leaf Node: label = 0
         |--> Child Value: 1
            Leaf Node: label = 0
         |--> Child Value: 2
            Leaf Node: label = 0
         |--> Child Value: 3
            Leaf Node: label = 0
         |--> Child Value: 4
            

**<font color = blue size=4>第三部分:作业提交</font>**

<span style="color:purple">本次实验分两周完成,实验报告提交地址(学号+姓名+实验五):https://send2me.cn/s0Yr-w4z/QCKd4ibz34miqQ   截止日期:10.20 14:20</span>

<span style="color:purple">实验课件获取地址:https://www.jianguoyun.com/p/DZRe9cwQp5WhChjz4p8FIAA</span>